In [305]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

In [306]:
df = pd.read_csv('data_treated.csv')
df

,Run,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,...,xNH3_cal,xN2_cal,xH2_cal,Fin [mmol/min],Fout [mmol/min],Vin [mL/min],Vout [mL/min],Vavg [mL/min],Vcat [mL],rt [min]
0,0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,...,0.925,0.01875,0.05625,1.226182,1.273955,57.670636,59.917543,58.786933,0.402685,0.005586
1,1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,...,0.948,0.01300,0.03900,2.452364,2.517827,115.341271,118.420196,116.873975,0.402685,0.002793
2,2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,...,0.951,0.01225,0.03675,3.678546,3.770934,173.011907,177.357157,175.175550,0.402685,0.001862
3,3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,...,0.795,0.05125,0.15375,1.226182,1.366219,62.701660,69.862574,66.217597,0.402685,0.005138
4,4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,...,0.843,0.03925,0.11775,2.452364,2.661274,125.403320,136.086078,130.671929,0.402685,0.002569
5,5,0.3,350,90,3.678546,0.875,0.031,0.094,0.066667,3.433310,...,0.875,0.03125,0.09375,3.678546,3.923782,188.104981,200.645313,194.307707,0.402685,0.001713
6,6,0.3,400,30,1.226182,0.514,0.122,0.365,0.321004,0.832573,...,0.514,0.12150,0.36450,1.226182,1.619791,67.732685,89.475145,78.100155,0.402685,0.004756
7,7,0.3,400,60,2.452364,0.647,0.088,0.265,0.214329,1.926751,...,0.647,0.08825,0.26475,2.452364,2.977977,135.465370,164.499538,149.512899,0.402685,0.002378
8,8,0.3,400,90,3.678546,0.713,0.072,0.216,0.167542,3.062234,...,0.713,0.07175,0.21525,3.678546,4.294858,203.198055,237.242329,219.780910,0.402685,0.001585
9,9,0.3,450,30,1.226182,0.202,0.200,0.599,0.663894,0.412128,...,0.202,0.19950,0.59850,1.226182,2.040236,72.763710,121.071064,94.876506,0.402685,0.004427


In [307]:
df_rx = pd.DataFrame()
df_rx['Temp [K]'] = df['Temp [degC]'] + 273.15
df_rx['F0 [mmol/min]'] = df['Fin [mmol/min]']
df_rx['rt [min]'] = df['rt [min]']
df_rx['F [mmol/min]'] = df['Fin [mmol/min]']*(1-df['Conv'])
df_rx['W_F [g-min/mmol]'] = df['Wcat [g]']/df['Fin [mmol/min]']

df_rx

,Temp [K],F0 [mmol/min],rt [min],F [mmol/min],W_F [g-min/mmol]
0,573.15,1.226182,0.005586,1.178409,0.244662
1,573.15,2.452364,0.002793,2.386900,0.122331
2,573.15,3.678546,0.001862,3.586158,0.081554
3,623.15,1.226182,0.005138,1.086144,0.244662
4,623.15,2.452364,0.002569,2.243454,0.122331
5,623.15,3.678546,0.001713,3.433310,0.081554
6,673.15,1.226182,0.004756,0.832573,0.244662
7,673.15,2.452364,0.002378,1.926751,0.122331
8,673.15,3.678546,0.001585,3.062234,0.081554
9,723.15,1.226182,0.004427,0.412128,0.244662


In [308]:
class NH3_decomposition:
    def __init__(self, data_df, Temp):
        self.data = df_rx[df_rx['Temp [K]'] == Temp + 273.15].copy()
        self.T = Temp + 273.15
        self.P = 1 # atm

        self.data = self.data.sort_values(by='rt [min]').reset_index(drop=True)

        self.RT = self.data['rt [min]'].values
        self.F0 = self.data['F0 [mmol/min]'].values
        self.nNH3 = self.data['F [mmol/min]'].values

    def partial_pressure(self, N, N0):
        nNH3 = N
        nN2 = 0.5*(N0-N)
        nH2 = 1.5*(N0-N)
        pNH3 = self.P*nNH3/(nNH3+nN2+nH2)
        pN2 = self.P*nN2/(nNH3+nN2+nH2)
        pH2 = self.P*nH2/(nNH3+nN2+nH2)

        return np.array([pNH3, pN2, pH2])
    
    def rate_equation(self, N, k, N0):
        p = self.partial_pressure(N, N0) + 1e-6
        Keq = -0.36456 + 0.000371 * (self.T - 273.15) + 91.85171/(self.T - 273.15)
        r = k[0]*((p[0]**2/(p[2]**3))**k[1] - (p[1]*Keq**2*(p[2]**3/p[0]**2)**(1-k[1])))
        return r

    def ode(self, t, y, k, N0):
        N = y[0]
        rNH3 = self.rate_equation(N, k, N0)
        dNH3dt = -rNH3*0.3
        return [dNH3dt]

    def solve_ode(self, k, N0, tend):
        y0 = [N0]  # 초기값은 리스트로, N0는 스칼라로 전달
        t_span = (0, tend)
        sol = solve_ivp(
            fun = lambda t, y: self.ode(t, y, k, N0),  # N0는 스칼라
            t_span = t_span,
            y0 = y0,
            t_eval = [tend],
            method = 'LSODA',
            rtol = 1e-5,
            atol = 1e-5
        )

        return sol.y[0]
    
    def obj_fun(self, params):
        k = params
        err = 0
        for i in range(len(self.RT)):
            N0 = self.F0[i]
            tend = self.RT[i]
            N_pred = self.solve_ode(k, N0, tend)
            err += np.sum((self.nNH3[i] - N_pred[0])**2)
        return err
    
    def est_params(self):
        x0 = np.array([10000,0.5])
        bounds = [(0,10000000),(0,1)]
        best_x = None
        best_loss = float('inf')

        for i in range(100):  # 100번 → 5번으로 줄임 (테스트용)
            print(f'시도 {i+1} 시작')
            
            # minimize 함수 내에서 10번마다 현재 몇 번째 iteration인지 출력하는 콜백 함수 정의
            def print_every_10(xk):
                if not hasattr(print_every_10, "iter"):
                    print_every_10.iter = 0
                print_every_10.iter += 1
                if print_every_10.iter % 10 == 0:
                    print(f"minimize 내부 {print_every_10.iter}번째 iteration 진행 중")
                    
            res = minimize(
                fun = self.obj_fun,
                x0 = x0,
                method = 'L-BFGS-B',
                bounds = bounds,
                callback = print_every_10,
                options={'maxiter': 100}
            )
            
            # 현재 시도의 loss 출력
            print(f'시도 {i+1} 완료 - Loss: {res.fun:.6f}, Best Loss: {best_loss:.6f}, Best x: {best_x}')

            if res.fun < best_loss:
                best_loss = res.fun
                best_x = res.x.copy()
                x0 = res.x.copy()
            else:
                x0 = best_x.copy() if best_x is not None else x0

            # x0에 랜덤 변화 추가
            x0 = x0 * (1 + np.random.randn(len(x0)) * 1)

        return best_x

In [309]:
a = NH3_decomposition(df_rx, 450)
print("클래스 생성 완료")

클래스 생성 완료


In [310]:
best_params = a.est_params()
print(f"최종 결과: {best_params}")

시도 1 시작
시도 1 완료 - Loss: 0.006111, Best Loss: inf, Best x: None
시도 2 시작
시도 2 완료 - Loss: 0.211310, Best Loss: 0.006111, Best x: [5.37440665e+02 4.45018270e-01]
시도 3 시작
minimize 내부 10번째 iteration 진행 중
minimize 내부 20번째 iteration 진행 중
minimize 내부 30번째 iteration 진행 중
시도 3 완료 - Loss: 0.000068, Best Loss: 0.006111, Best x: [5.37440665e+02 4.45018270e-01]
시도 4 시작
minimize 내부 10번째 iteration 진행 중
minimize 내부 20번째 iteration 진행 중
시도 4 완료 - Loss: 0.000068, Best Loss: 0.000068, Best x: [413.24643558   0.54488755]
시도 5 시작
minimize 내부 10번째 iteration 진행 중
minimize 내부 20번째 iteration 진행 중
시도 5 완료 - Loss: 0.000068, Best Loss: 0.000068, Best x: [413.20464875   0.54493706]
시도 6 시작
시도 6 완료 - Loss: 2.932092, Best Loss: 0.000068, Best x: [413.21299619   0.54492845]
시도 7 시작
시도 7 완료 - Loss: 3.091090, Best Loss: 0.000068, Best x: [413.21299619   0.54492845]
시도 8 시작
minimize 내부 10번째 iteration 진행 중
minimize 내부 20번째 iteration 진행 중
시도 8 완료 - Loss: 0.000068, Best Loss: 0.000068, Best x: [413.21299619   0.54492845]
시도 9

In [311]:
best_params

array([413.21779168,   0.54492368])

In [312]:
from curses import reset_prog_mode


test_df = df[df['Temp [degC]'] == 450]
test_df
k_best = best_params
def ode2(t,N):
    k = k_best

    pNH3 = N[0]/sum(N)+1e-6
    pN2 = N[1]/sum(N)+1e-6
    pH2 = N[2]/sum(N)+1e-6
    Keq = -0.36456 + 0.000371 * (450 - 273.15) + 91.85171/(450 - 273.15)
    r = k[0]*((pNH3**2/pH2**3)**k[1] - pN2*Keq**2*(pH2**3/pNH3**2)**(1-k[1]))

    dNH3 = -r*0.3
    dN2 = 0.5*r*0.3
    dH2 = 1.5*r*0.3

    return [dNH3,dN2,dH2]

idx = 0
N0 = np.array([test_df.iloc[idx]['Fin [mmol/min]'],0,0])
res = solve_ivp(ode2,[0,test_df.iloc[idx]['rt [min]']],N0,method='LSODA')

res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  4.129e-19 ...  4.291e-03  4.427e-03]
        y: [[ 1.226e+00  1.226e+00 ...  4.187e-01  4.122e-01]
            [ 0.000e+00  1.172e-07 ...  4.037e-01  4.070e-01]
            [ 0.000e+00  3.515e-07 ...  1.211e+00  1.221e+00]]
      sol: None
 t_events: None
 y_events: None
     nfev: 348
     njev: 0
      nlu: 0

In [313]:
df[['molF_NH3','molF_N2','molF_H2','rt [min]']]

,molF_NH3,molF_N2,molF_H2,rt [min]
0,1.178409,0.023887,0.071660,0.005586
1,2.386900,0.032732,0.098195,0.002793
2,3.586158,0.046194,0.138582,0.001862
3,1.086144,0.070019,0.210056,0.005138
4,2.243454,0.104455,0.313365,0.002569
5,3.433310,0.122618,0.367855,0.001713
6,0.832573,0.196805,0.590414,0.004756
7,1.926751,0.262806,0.788419,0.002378
8,3.062234,0.308156,0.924468,0.001585
9,0.412128,0.407027,1.221081,0.004427


In [314]:
(N0[0]-res.y[0][-1])/N0[0]

np.float64(0.6638740459891541)

In [315]:
nNH3 = res.y[0][-1]
nN2 = res.y[1][-1]
nH2 = res.y[2][-1]

n_total = nNH3 + nN2 + nH2

pNH3 = nNH3/n_total
pN2 = nN2/n_total
pH2 = nH2/n_total

print(pNH3, pN2, pH2)

0.20201406159384105 0.19949648460153982 0.5984894538046192


In [316]:
X = (N0[0] - nNH3)/N0[0]
y_NH3 = (1-X)/(1+X)
y_N2 = 0.5*X/(1+X)
y_H2 = 1.5*X/(1+X)

y_NH3, y_N2, y_H2

(np.float64(0.2020140615938407),
 np.float64(0.19949648460153982),
 np.float64(0.5984894538046195))

In [317]:

# best_x를 npy 파일로 저장합니다.
np.save('best_k0.npy', best_params)
